<a href="https://colab.research.google.com/github/Thiraput01/QwenMed/blob/main/notebooks/QwenMed_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libs and set env

In [ ]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
!pip install --no-deps unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.4/147.4 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux"

In [ ]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ["HF_TOKEN"] = user_secrets.get_secret("HF_TOKEN")
os.environ["WANDB_API_KEY"] = user_secrets.get_secret("WANDB_API_KEY")

# Prepare Datasets for fine-tuning

In [ ]:
from datasets import load_dataset
reasoning_ds = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en", split="train")
non_reasoning_ds = load_dataset("Laurent1/MedQuad-MedicalQnADataset_128tokens_max", split="train")

README.md:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

medical_o1_sft.json:   0%|          | 0.00/58.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19704 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

questions-reponses_medecales.csv:   0%|          | 0.00/7.31M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
reasoning_ds, reasoning_ds[0]

(Dataset({
     features: ['Question', 'Complex_CoT', 'Response'],
     num_rows: 19704
 }),
 {'Question': 'Given the symptoms of sudden weakness in the left arm and leg, recent long-distance travel, and the presence of swollen and tender right lower leg, what specific cardiac abnormality is most likely to be found upon further evaluation that could explain these findings?',
  'Complex_CoT': "Okay, let's see what's going on here. We've got sudden weakness in the person's left arm and leg - and that screams something neuro-related, maybe a stroke?\n\nBut wait, there's more. The right lower leg is swollen and tender, which is like waving a big flag for deep vein thrombosis, especially after a long flight or sitting around a lot.\n\nSo, now I'm thinking, how could a clot in the leg end up causing issues like weakness or stroke symptoms?\n\nOh, right! There's this thing called a paradoxical embolism. It can happen if there's some kind of short circuit in the heart - like a hole that should

In [ ]:
non_reasoning_ds, non_reasoning_ds[0]

(Dataset({
     features: ['text'],
     num_rows: 15549
 }),
 {'text': 'Below is an instruction from Human. Write a response.\n    ### Instruction:\n    Who is at risk for Lymphocytic Choriomeningitis (LCM)? ?\n    ### Response:\n    LCMV infections can occur after exposure to fresh urine, droppings, saliva, or nesting materials from infected rodents.  Transmission may also occur when these materials are directly introduced into broken skin, the nose, the eyes, or the mouth, or presumably, via the bite of an infected rodent.\n    '})

# Load Model and Preprocess

In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-1.7B-unsloth-bnb-4bit",
    max_seq_length = 1024,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-06-16 04:13:35.240343: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750047215.439969      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750047215.500155      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.2: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 6.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.41G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 64,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
    use_rslora = True,
    loftq_config = None,
)

Unsloth 2025.6.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
def generate_conversation_normal(batch):
    instructions = batch["text"]
    new_conversations = []

    for text in instructions:
        try:
            parts = text.split("### Instruction:")
            instruction_and_response = parts[1]

            question_part, answer_part = instruction_and_response.split("### Response:")

            question = question_part.strip()
            answer = answer_part.strip()

            new_conversations.append([
                {"role": "user", "content": question},
                {"role": "assistant", "content": answer},
            ])
        except (IndexError, ValueError):
            print(f"Skipping row: {text}")
            continue

    return {"conversations": new_conversations}


def generate_conversation_reasoning(batch):
    out = []
    for q, cot, ans in zip(
            batch["Question"],
            batch["Complex_CoT"],
            batch["Response"]):
        assistant_text = f"<think>\n{cot}\n</think>\n\n{ans}"
        out.append([
            {"role": "user",      "content": q},
            {"role": "assistant", "content": assistant_text},
        ])
    return {"conversations": out}

In [ ]:
normal_conversations = tokenizer.apply_chat_template(
    non_reasoning_ds.map(generate_conversation_normal, batched = True)["conversations"],
    tokenize = False,
)

Map:   0%|          | 0/15549 [00:00<?, ? examples/s]

In [ ]:
reasoning_conversations = tokenizer.apply_chat_template(
    reasoning_ds.map(generate_conversation_reasoning, batched = True)["conversations"],
    tokenize = False,
)

Map:   0%|          | 0/19704 [00:00<?, ? examples/s]

In [ ]:
print(normal_conversations[0])

<|im_start|>user
Who is at risk for Lymphocytic Choriomeningitis (LCM)? ?<|im_end|>
<|im_start|>assistant
<think>

</think>

LCMV infections can occur after exposure to fresh urine, droppings, saliva, or nesting materials from infected rodents.  Transmission may also occur when these materials are directly introduced into broken skin, the nose, the eyes, or the mouth, or presumably, via the bite of an infected rodent.<|im_end|>



In [ ]:
print(reasoning_conversations[0])

<|im_start|>user
Given the symptoms of sudden weakness in the left arm and leg, recent long-distance travel, and the presence of swollen and tender right lower leg, what specific cardiac abnormality is most likely to be found upon further evaluation that could explain these findings?<|im_end|>
<|im_start|>assistant
<think>
Okay, let's see what's going on here. We've got sudden weakness in the person's left arm and leg - and that screams something neuro-related, maybe a stroke?

But wait, there's more. The right lower leg is swollen and tender, which is like waving a big flag for deep vein thrombosis, especially after a long flight or sitting around a lot.

So, now I'm thinking, how could a clot in the leg end up causing issues like weakness or stroke symptoms?

Oh, right! There's this thing called a paradoxical embolism. It can happen if there's some kind of short circuit in the heart - like a hole that shouldn't be there.

Let's put this together: if a blood clot from the leg somehow 

In [ ]:
len(normal_conversations), len(reasoning_conversations)

(15549, 19704)

# Select 80% Reasoning and 20% Normal

In [ ]:
import pandas as pd

total_samples = 12000
eval_ratio = 0.15

normal_total = int(0.2 * total_samples)
reasoning_total = int(0.8 * total_samples)

normal_eval_size = int(normal_total * eval_ratio)
reasoning_eval_size = int(reasoning_total * eval_ratio)

normal_train_size = normal_total - normal_eval_size
reasoning_train_size = reasoning_total - reasoning_eval_size

normal_df = pd.Series(normal_conversations)
reasoning_df = pd.Series(reasoning_conversations)

In [ ]:
normal_sample = normal_df.sample(n=normal_total, random_state=42)
reasoning_sample = reasoning_df.sample(n=reasoning_total, random_state=42)

In [ ]:
normal_train = normal_sample.iloc[:normal_train_size]
normal_eval = normal_sample.iloc[normal_train_size:]


reasoning_train = reasoning_sample.iloc[:reasoning_train_size]
reasoning_eval = reasoning_sample.iloc[reasoning_eval_size:]

In [ ]:
from datasets import Dataset

train_data = pd.concat([normal_train, reasoning_train])
train_data.name = "text"
train_dataset = Dataset.from_pandas(pd.DataFrame(train_data))
train_dataset = train_dataset.shuffle(seed=42)


eval_data = pd.concat([normal_eval, reasoning_eval])
eval_data.name = "text"
eval_dataset = Dataset.from_pandas(pd.DataFrame(eval_data))
eval_dataset = eval_dataset.shuffle(seed=43)

In [ ]:
len(train_dataset), len(eval_dataset)

(10200, 8520)

In [ ]:
len(normal_train), len(reasoning_train)

(2040, 8160)

In [ ]:
train_dataset, train_dataset[0]

(Dataset({
     features: ['text', '__index_level_0__'],
     num_rows: 10200
 }),
 {'text': "<|im_start|>user\nA 44-year-old woman with rheumatoid arthritis develops painful oral ulcers and elevated liver enzymes several weeks after starting a new medication. The medication's side effect profile and the lab findings suggest a specific drug action. What is the primary mechanism of action of this drug used to treat her rheumatoid arthritis?<|im_end|>\n<|im_start|>assistant\n<think>\nAlright, here's a situation. A 44-year-old woman has rheumatoid arthritis. Now, she starts experiencing painful oral ulcers and her liver enzymes are on the rise a few weeks after kicking off a new medication. These are some pretty specific red flags.\n\nHmm, let's think. Oral ulcers and liver issues sound familiar, especially if she's taking something new for her rheumatoid arthritis. That's where methotrexate comes into play. It's a common drug for treating that condition and these side effects line up wit

# Train the model

In [ ]:
OUTPUT_DIR = "qwen3_med_lora_adapter"
HF_REPO    = "Thiraput01/QwenMed-1.7B-Reasoning"

In [ ]:
from trl import SFTTrainer, SFTConfig
import wandb

training_args = SFTConfig(
        dataset_text_field = "text",

        per_device_train_batch_size = 2,
        per_device_eval_batch_size = 4,
        gradient_accumulation_steps = 8,

        warmup_ratio  = 0.1,
        num_train_epochs = 2,
        learning_rate = 5e-5,
        logging_steps = 10,
        eval_steps = 300,
        save_steps = 300,
        eval_strategy = "steps",
        save_strategy = "steps",
        load_best_model_at_end = True,
        metric_for_best_model = "eval_loss",
        greater_is_better = False,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 42,
        push_to_hub = True,
        hub_strategy = "checkpoint",
        hub_model_id = HF_REPO,
        report_to = "wandb",
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    args = training_args,
)

wandb.init(
    project = "qwen3-med",
    name    = "qlora-1.7b-run4",
    config  = {
        "model": "Qwen3-1.7B",
        "lora_r": 32,
        "lr": 5e-5,
        "batch_size": 2,
        "num_epochs": 1,
      },
)

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/10200 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/8520 [00:00<?, ? examples/s]

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: thiraputball (thiraputball-chulalongkorn-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.19.9
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250616_041500-vdw5cgdp
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run qlora-1.7b-run4
wandb: ⭐️ View project at https://wandb.ai/thiraputball-chulalongkorn-university/qwen3-med
wandb: 🚀 View run at https://wandb.ai/thiraputball-chulalongkorn-university/qwen3-med/runs/vdw5cgdp


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,200 | Num Epochs = 2 | Total steps = 1,274
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 34,865,152/1,700,000,000 (2.05% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
300,1.531300,1.494739
600,1.461700,1.429246
900,1.374300,1.388162
1200,1.361300,1.376883


Unsloth: Not an error, but Qwen3ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient
